In [2]:
import pymysql
import numpy as np
import pandas as pd
import pandas as pd
import os, json, time
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
from sqlalchemy import create_engine
pymysql.install_as_MySQLdb()

In [3]:
filename = 'https://datasets.imdbws.com/title.basics.tsv.gz'
df = pd.read_csv(filename, sep='\t', low_memory=False)

df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [4]:
df.dropna(inplace=True)

In [5]:
df = df.loc[(df['startYear'] >= '2000')]

In [6]:
df = df.replace({'\\N':np.nan})

In [14]:
df.dropna(inplace = True)
df.head(100)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
108563,tt0111056,tvSeries,Gensomaden Saiyuki,Gensomaden Saiyuki,0,2000,2001,23,"Action,Adventure,Animation"
137836,tt0142032,tvMiniSeries,Dune,Dune,0,2000,2000,265,"Adventure,Drama,Sci-Fi"
153556,tt0158466,tvMiniSeries,Anne of Green Gables: The Continuing Story,Anne of Green Gables: The Continuing Story,0,2000,2000,185,"Drama,Family,Romance"
155749,tt0160904,tvSeries,MI-5,Spooks,0,2002,2011,60,"Action,Crime,Drama"
158616,tt0163944,tvMiniSeries,Labyrinten,Labyrinten,0,2000,2000,259,"Drama,Thriller"
...,...,...,...,...,...,...,...,...,...
223457,tt0233307,tvMiniSeries,Beautiful Life,Beautiful Life,0,2000,2000,60,"Drama,Romance"
223638,tt0233491,tvSeries,Cover Me: Based on the True Life of an FBI Family,Cover Me: Based on the True Life of an FBI Family,0,2000,2001,60,"Crime,Drama,Family"
224385,tt0234265,tvMiniSeries,Monsignor Renard,Monsignor Renard,0,2000,2000,295,"Drama,War"
225236,tt0235137,tvSeries,Son of the Beach,Son of the Beach,0,2000,2002,22,Comedy


In [12]:
df.loc[(df['titleType'] == 'Movie')]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres


In [15]:
file_2 = 'https://datasets.imdbws.com/title.akas.tsv.gz'
akas = pd.read_csv(file_2, sep='\t', low_memory=False)

In [19]:
akas = akas.replace({'\\N':np.nan})

In [20]:
akas.loc[(akas['region'] == 'US')]

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0
...,...,...,...,...,...,...,...,...
34196883,tt9916702,1,Loving London: The Playground,US,NaN,NaN,NaN,0
34196920,tt9916720,10,The Demonic Nun,US,NaN,tv,NaN,0
34196922,tt9916720,12,The Nun 2,US,NaN,imdbDisplay,NaN,0
34196940,tt9916756,1,Pretty Pretty Black Girl,US,NaN,imdbDisplay,NaN,0


In [21]:
is_documentary = df['genres'].str.contains('documentary',case=False)
df = df[~is_documentary]

In [22]:
keepers =df['tconst'].isin(akas['titleId'])
keepers

108563     True
137836     True
153556     True
155749     True
158616     True
           ... 
9452325    True
9452358    True
9452459    True
9452665    True
9452845    True
Name: tconst, Length: 23400, dtype: bool

In [23]:
df = df[keepers]
df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
108563,tt0111056,tvSeries,Gensomaden Saiyuki,Gensomaden Saiyuki,0,2000,2001,23,"Action,Adventure,Animation"
137836,tt0142032,tvMiniSeries,Dune,Dune,0,2000,2000,265,"Adventure,Drama,Sci-Fi"
153556,tt0158466,tvMiniSeries,Anne of Green Gables: The Continuing Story,Anne of Green Gables: The Continuing Story,0,2000,2000,185,"Drama,Family,Romance"
155749,tt0160904,tvSeries,MI-5,Spooks,0,2002,2011,60,"Action,Crime,Drama"
158616,tt0163944,tvMiniSeries,Labyrinten,Labyrinten,0,2000,2000,259,"Drama,Thriller"
...,...,...,...,...,...,...,...,...,...
9452325,tt9915022,tvSeries,Yarali Kuslar,Yarali Kuslar,0,2019,2019,90,Drama
9452358,tt9915114,tvSeries,Touche pas à mon sport,Touche pas à mon sport,0,2015,2016,65,"Sport,Talk-Show"
9452459,tt9915338,tvSeries,Aunty Donna: Camp Bush Camp!,Aunty Donna: Camp Bush Camp!,0,2018,2018,5,Comedy
9452665,tt9915822,tvSeries,Ichhapyaari Naagin,Ichhapyaari Naagin,0,2016,2017,20,Fantasy


In [24]:
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

[]

In [25]:
df.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [26]:
df = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0111056,tvSeries,Gensomaden Saiyuki,Gensomaden Saiyuki,0,2000,2001,23,"Action,Adventure,Animation"
1,tt0142032,tvMiniSeries,Dune,Dune,0,2000,2000,265,"Adventure,Drama,Sci-Fi"
2,tt0158466,tvMiniSeries,Anne of Green Gables: The Continuing Story,Anne of Green Gables: The Continuing Story,0,2000,2000,185,"Drama,Family,Romance"
3,tt0160904,tvSeries,MI-5,Spooks,0,2002,2011,60,"Action,Crime,Drama"
4,tt0163944,tvMiniSeries,Labyrinten,Labyrinten,0,2000,2000,259,"Drama,Thriller"
